# 1.	参照例2完成：在 www.amazon.cn 搜索python，提取搜索页面python书籍的价格和名称，至少3个页面的信息

**+ 注意是书籍而非课程；**


In [6]:
# 定义所需类

import requests
from bs4 import BeautifulSoup
import re

class amazon_python(object):
    
    def __init__(self,depth = 2):
        self.url = 'http://www.amazon.cn/s'
        self.depth = depth
        self.count = 1
        self.headers = {
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)',
            'Host': 'www.amazon.cn'
        }



    def get_data(self,url,page):
        self.params = {
            'k': 'python',
            'i': 'stripbooks',
            'page': str(page),
        }
        # try:
        response = requests.get(url=self.url,timeout = 20,params=self.params,headers = self.headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        
        return response.text
        # except:
        #     return''
        
        
        
        
    def parse_data(self,data,result):
        bs = BeautifulSoup(data,"html.parser")
        table = bs.find(name='span',attrs={'data-component-type':"s-search-results"}).find(name='div',attrs={"class":"s-main-slot s-result-list s-search-results sg-row"}).find_all('div',attrs = {'data-index':re.compile(r'\d+')})
        # print(table)
        for list in table:
            temp = {}
            try:
                
                temp['price']=list.find(name = 'span',attrs = {'class':"a-price"}).find(name = 'span',attrs = {'class':"a-offscreen"}).string
                temp['name']=list.find(name='span',attrs={'class':"a-size-medium a-color-base a-text-normal"}).string
                temp['rank']=self.count
                self.count += 1 
                
                result.append(temp)
            except:
                pass
            
        return result
        
        

    def printlist(self,list):
        f = '{:^4}\t{:^16}\t{:^32}'
        print(f.format('序号','价格','商品名称',chr(12888)))
        for i in range(len(list)):
            print(f.format(list[i]['rank'],list[i]['price'],list[i]['name'],chr(12888)))
            
        
        
        
        
    def main(self):
        result = []
        for i in range(self.depth):
            data = self.get_data(url=self.url,page=i+1)
            result = self.parse_data(data,result)
            
        self.printlist(result)
    

In [7]:
# 实例化并运行

if __name__ == '__main__':
    trial = amazon_python(depth = 3)
    trial.main()

 序号 	       价格       	              商品名称              
 1  	     ¥0.00      	      编程小白的第一本 Python 入门书       
 2  	     ¥24.60     	Python编程：从入门到实践（第2版）（新版上线！！！！沿袭第1版讲解清晰透彻、循序渐进的特点，并全面升级！源代码文件+视频讲解+配套编程环境，零基础自学Python一本就够！）（图灵图书）
 3  	     ¥10.99     	  Python编程快速上手——让繁琐工作自动化（异步图书）  
 4  	     ¥84.74     	Python编程实战（妙趣横生的项目编程，不同凡响的Python之旅）
 5  	     ¥55.99     	      流畅的Python (图灵程序设计丛书)      
 6  	     ¥49.90     	       Python编程轻松进阶（图灵图书）       
 7  	     ¥70.00     	       算法深潜：勇敢者的Python探险        
 8  	     ¥44.90     	 Excel+Python：飞速搞定数据分析与处理（图灵图书） 
 9  	     ¥39.99     	        Python深度学习（图灵图书）        
 10 	     ¥71.06     	   Python工匠：案例、技巧与工程实践（图灵图书）    
 11 	     ¥38.50     	Python深度学习入门：从零构建CNN和RNN【前Facebook数据科学家带你领略深度学习领域的全貌!】（图灵图书）
 12 	    ¥118.30     	        Python王者归来（增强版）         
 13 	     ¥26.99     	 Python带我起飞——入门、进阶、商业实战（博文视点图书） 
 14 	     ¥67.74     	     IT审计：用SQL+Python提升工作效率     
 15 	     ¥70.00     	   Python黑帽子：黑客与渗透测试编程之道（第2版）   
 16 	     ¥41.00     	Pyt

# 2． 参照ppt中的例子，采用scrapy 实现爬取股票数据